# Output Parser | Listing

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from dotenv import dotenv_values

env_values = dotenv_values(".env")

OPENAI_API_KEY = env_values["OPEN_ROUTER_API_KEY"]
OPENAI_API_BASE = env_values["OPENAI_API_BASE"]
MODEL_NAME = "openai/gpt-5-mini"   

In [2]:
llm = ChatOpenAI(
    model=MODEL_NAME,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base = OPENAI_API_BASE,
    temperature=0.7
)

In [3]:
task = "List three popular Egyptian plates."

print(llm.invoke(task).content)

- Koshari (koshary) — a hearty street-food mix of rice, macaroni, lentils, chickpeas, spiced tomato sauce and fried onions.  
- Ful medames — slow-cooked fava beans mashed or whole, typically served for breakfast with olive oil, lemon, cumin and bread.  
- Molokhia (mulukhiyah) — a green jute-leaf stew flavored with garlic and coriander, usually served over rice or with bread and chicken or rabbit.


In [4]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [5]:
prompt = PromptTemplate(
    template= "List three popular {type} plates.\n{format_instructions}",
    input_variables=['type'],
    partial_variables={"format_instructions" : format_instructions}
)

demo_msg = prompt.format(type = 'Egyptian')
print(demo_msg)

List three popular Egyptian plates.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [6]:
print(llm.invoke(demo_msg).content)

Koshari, Ful medames, Molokhia


In [7]:
response = llm.invoke(demo_msg).content
output = output_parser.parse(response)
output

['Koshary', 'Ful medames', 'Molokhia']

# Custom Output Parser | Listing 

In [8]:
format_instruction = "Replay with a datetime format Day/Month/Year like '12/2/2004'."

prompt = PromptTemplate(
    template= "when was {person} born\n{format_instruction}",
    input_variables=['person','format_instruction']
)

message = prompt.format(person = "Ahmed Zewail", format_instruction = format_instruction)

print(llm.invoke(message).content)

26/2/1946


# PyDantic Magic Parser | Listing

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

class Plate(BaseModel):
    plate_name: str = Field(description='name of the plate')
    ingredients: List[str] = Field(description='list of names of the ingredients')
    
parser = PydanticOutputParser(pydantic_object=Plate)

format_instruction = parser.get_format_instructions()

prompt = PromptTemplate(
    template= 'Answer the query.\n{plate_name}\n{format_instruction}', 
    input_variables=['plate_name'],
    partial_variables={'format_instruction' : format_instruction}
)

plate_name = 'Shawerma'

msg = prompt.format_prompt(plate_name = plate_name)

print(llm.invoke(msg).content)

{
  "plate_name": "Shawerma",
  "ingredients": [
    "marinated sliced chicken (or beef/lamb)",
    "pita bread or flatbread",
    "toum (garlic sauce) or tahini",
    "tomatoes",
    "cucumbers",
    "pickles (cucumber or turnip)",
    "lettuce",
    "onions",
    "shawarma spice mix (paprika, cumin, coriander, turmeric)",
    "olive oil",
    "lemon juice"
  ]
}


In [14]:
plate_name = 'كفته داوود'

msg = prompt.format_prompt(plate_name = plate_name)

print(llm.invoke(msg).content)

{
  "plate_name": "كفته داوود",
  "ingredients": [
    "لحم مفروم (بقري أو غنم)",
    "بصل مفرور",
    "ثوم مفروم",
    "بقدونس مفروم",
    "بيضة",
    "فتات خبز (بقسماط) أو خبز مبلل ومعصور",
    "ملح",
    "فلفل أسود",
    "بهارات مشكلة (مثل قرفة أو بهار حلو أو كمون)",
    "زيت نباتي أو زيت زيتون",
    "طماطم مفرومة أو صلصة طماطم",
    "معجون طماطم",
    "ماء أو مرق",
    "سكر قليل (لموازنة حموضة الطماطم)",
    "عصير ليمون أو خل (اختياري)",
    "بطاطس مقطعة أو مطبوخة (اختياري)"
  ]
}
